In [ ]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

import missingno as msno
import pandas_profiling

from src.visualization.visualize import distplot

sns.set(font_scale=1.)

In [ ]:
import folium
from folium.plugins import FastMarkerCluster

## Chargement des données

In [ ]:
data = dict()
for dirname, _, filenames in os.walk('../data/raw'):
    for filename in filenames:
        key = filename.split('.')[0]
        key = '_'.join(key.split('_')[1:-1]) if key.startswith('olist') else key
        data[key] = pd.read_csv(os.path.join(dirname, filename))

In [ ]:
print(data.keys())

## Analyse des données géographiques

On commence par agréger les dataframes contenant d'une part les données utilisateur et d'autre part les données
contenant les localisations.

On a alors un dataframe contentant les toutes les positions connues et pour tout les utilisateurs

In [ ]:
loc_df = pd.merge(data['customers'],
                  data['geolocation'],
                  left_on='customer_zip_code_prefix',
                  right_on='geolocation_zip_code_prefix')
print(loc_df.shape)

On peut réduire la quantité de donnée en ne gardant que la moyenne des positions (latitude et longitude) connues
des utilisateurs.

In [ ]:
mean_loc_df = loc_df.groupby('customer_unique_id')[
    ['geolocation_lat', 'geolocation_lng']].mean()
print(mean_loc_df.shape)

In [ ]:
gdf_customer = gpd.GeoDataFrame(mean_loc_df.copy(),
                                geometry=gpd.points_from_xy(mean_loc_df['geolocation_lat'],
                                                            mean_loc_df['geolocation_lng']))

In [ ]:
gdf_customer.reset_index(inplace=True)

In [ ]:
gdf_customer.plot()

### Nombre de positions connues par client

In [ ]:
distplot(loc_df.groupby('customer_unique_id').count()['customer_id'])
plt.show()

**Hypothèse** On enregistre la position quand le client consulte l'application, donc le nombre de positions enregistrées par client traduit également la fréquence de consultation de l'application.

### Localisation de l'activité

In [ ]:
X = gdf_customer[['geolocation_lat', 'geolocation_lng']]

scaler = StandardScaler()
X = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=4)
kmeans.fit(X)

# silhouette_score(X, kmeans.labels_) # around 0.55

In [ ]:
gdf_customer['group'] = kmeans.labels_

In [ ]:
gdf_customer['group'].value_counts()

In [ ]:
centers = pd.DataFrame(scaler.inverse_transform(kmeans.cluster_centers_))
centers = pd.concat([gdf_customer['group'].value_counts().sort_index(),
                     centers], axis=1)

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 8))

sns.scatterplot(gdf_customer['geolocation_lat'],
                gdf_customer['geolocation_lng'],
                hue=kmeans.labels_, ax=ax,
                markers=False)


sns.scatterplot(x=0,
                y=1,
                size='group',
                sizes=(200, 2000),
                legend=False,
                ax=ax, data=centers, alpha=0.5)
plt.xlabel('lat')
plt.ylabel('long')
ax.set_xlim(10, -50)
ax.set_ylim(-30, -70)
plt.savefig('../reports/figures/localisation_clusters.png')

On garde également la déviation standard des positions de manière à savoir si le client voyage beaucoup

In [ ]:
std_loc_df = loc_df.groupby('customer_unique_id')[
    ['geolocation_lat', 'geolocation_lng']].std()
distplot(std_loc_df['geolocation_lat'])
distplot(std_loc_df['geolocation_lng'])
plt.show()

On peut visualiser la position des utlisateur sur une carte

In [ ]:
init_lat = mean_loc_df['geolocation_lat'].mean()
init_lng = mean_loc_df['geolocation_lng'].mean()

In [ ]:
                             
m = folium.Map(location=(mean_loc_df.mean()), zoom_start=3)
m.add_child(FastMarkerCluster(mean_loc_df.values.tolist()))

for center in range(centers.shape[0]):
    folium.CircleMarker(
        location=centers.iloc[center, 1:],
        radius=(centers.iloc[center, 0] // 2000).astype(str),
        fill=True,
        popup=center,
    ).add_to(m)

# m.save('../reports/customer_map_with_centroids.html')
m

In [ ]:
m.save('../reports/customer_map.html')

Un clustering des données géographique montre que la grande majorité de l'activité se concentre dans la région de Rio de Janeiro et São Paulo

Difficile de voir un intêret à utiliser les données géographiques à l'heure actuelle.

Peut-être regarder la distance moyenne par rapport aux vendeurs

### Positions des vendeurs

In [ ]:
seller_loc_df = pd.merge(data['sellers'],
                         data['geolocation'],
                         left_on='seller_zip_code_prefix',
                         right_on='geolocation_zip_code_prefix')
print(seller_loc_df.shape)

In [ ]:
seller_mean_loc_df = seller_loc_df.groupby('seller_id')[['geolocation_lat',
                                                         'geolocation_lng']].mean()
print(seller_mean_loc_df.shape)

On a donc 7 vendeurs pour lesquels on a pas de positions.

In [ ]:
seller_std_loc_df = seller_loc_df.groupby('seller_id')[['geolocation_lat',
                                                        'geolocation_lng']].std()
distplot(seller_std_loc_df['geolocation_lat'])
distplot(seller_std_loc_df['geolocation_lng'])
plt.show()

In [ ]:
init_lat = seller_mean_loc_df['geolocation_lat'].mean()
init_lng = seller_mean_loc_df['geolocation_lng'].mean()

In [ ]:
m = folium.Map(location=(seller_mean_loc_df.mean()), zoom_start=3)
m.add_child(FastMarkerCluster(seller_mean_loc_df.values.tolist()))
# m.save('../reports/seller_map.html')

## Retour aux reste des données 

In [ ]:
pd.merge(pd.merge(data['customers'], data['orders'], 
                  left_on='customer_id', 
                  right_on='customer_id'),
         pd.merge(data['orders'], data['order_payments'], 
                  left_on='order_id', 
                  right_on='order_id'),
         left_on='customer_id',
         right_on='customer_id',
        )

### Analyse des paiements

#### Types de paiments

In [ ]:
payment_type = pd.pivot_table(
    pd.merge(pd.merge(data['customers'], data['orders'],
                      left_on='customer_id',
                      right_on='customer_id'),
             pd.merge(data['orders'], data['order_payments'],
                      left_on='order_id',
                      right_on='order_id'),
             left_on='customer_id',
             right_on='customer_id',
             ), columns=['payment_type'], index=['customer_unique_id'], values=['payment_value'])

In [ ]:
payment_type.fillna(0, inplace=True)

In [ ]:
plt.bar(payment_type.sum().index.levels[1].values, payment_type.sum().values)
plt.title('Total CA par moyen de paiment')
plt.show()

In [ ]:
plt.bar(payment_type.sum().index.levels[1].values,
        (payment_type.sum().values / payment_type.sum().sum() * 100))
plt.title('Part CA par moyen de paiment')
plt.show()

#### Mensualités

In [ ]:
payment_installments = pd.merge(
    pd.merge(data['customers'], data['orders'],
             left_on='customer_id',
             right_on='customer_id'),
    pd.merge(data['orders'], data['order_payments'],
             left_on='order_id',
             right_on='order_id'),
    left_on='customer_id',
    right_on='customer_id',
).groupby('customer_unique_id').mean()['payment_installments']

In [ ]:
distplot(payment_installments, kde=False, norm_hist=True)
ax = plt.gca()
ax.set_title('Nombre de mensualités choisies par les clients')
plt.show()

In [ ]:
monthly_payment = pd.merge(
    pd.merge(data['customers'], data['orders'],
             left_on='customer_id',
             right_on='customer_id'),
    pd.merge(data['orders'], data['order_payments'],
             left_on='order_id',
             right_on='order_id'),
    left_on='customer_id',
    right_on='customer_id',
).groupby('customer_unique_id').agg({'payment_installments': np.mean,
                                     'payment_value': np.sum})

In [ ]:
monthly_payment['payment_installments'].replace(0, 1, inplace=True)

In [ ]:
monthly_payment['monthly_payment'] = monthly_payment['payment_value'] \
    / monthly_payment['payment_installments']

In [ ]:
distplot(monthly_payment['monthly_payment'], kde=False, norm_hist=True)
ax = plt.gca()
ax.set_title('Montant moyen des mensualités')
plt.show()

## Habitutes des clients

Part de CA en fonction du jour de la semaine

In [ ]:
orders = pd.merge(pd.merge(data['customers'], data['orders'],
                           left_on='customer_id',
                           right_on='customer_id'),
                  pd.merge(data['orders'], data['order_payments'],
                           left_on='order_id',
                           right_on='order_id'),
                  left_on='customer_id',
                  right_on='customer_id',
                  )

In [ ]:
orders['order_purchase_timestamp_x'] = pd.to_datetime(
    orders['order_purchase_timestamp_x'])
orders['weekday'] = orders['order_purchase_timestamp_x'].dt.dayofweek

In [ ]:
f, ax = plt.subplots(1)
ax.bar(orders.groupby('weekday').sum()['payment_value'].index.values,
       orders.groupby('weekday').sum()['payment_value'].values)
ax.set_xticks([x for x in range(7)])
ax.set_xticklabels(['lun', 'mar', 'mer', 'jeu', 'ven', 'sam', 'dim'])
# ax.set_title('Cumul CA en fonction du jour de la semaine')
plt.show()

Part de CA en fonction de l'heure

In [ ]:
orders['h'] = orders['order_purchase_timestamp_x'].dt.hour

In [ ]:
f, ax = plt.subplots(1)
ax.bar(orders.groupby('h').sum()['payment_value'].index.values,
       orders.groupby('h').sum()['payment_value'].values)
# ax.set_title("Cumul CA en fonction de l'heure")
plt.show()

In [ ]:
orders['d'] = orders['order_purchase_timestamp_x'].dt.day

In [ ]:
f, ax = plt.subplots(1)
ax.bar(orders.groupby('d').sum()['payment_value'].index.values,
       orders.groupby('d').sum()['payment_value'].values)
# ax.set_title('Cumul CA en fonction du jour du mois')
plt.show()

## Activité par région

Quelles régions rapportent le plus? 

In [ ]:
f, ax = plt.subplots(1, figsize=(12, 8))
ax.barh(orders.groupby('customer_state').sum()['payment_value'].sort_values().index.values,
        orders.groupby('customer_state').sum()['payment_value'].sort_values().values)
plt.title('Total CA en fonction des régions')
plt.show()

In [ ]:
orders.head()

In [ ]:
ts = orders.set_index(pd.to_datetime(orders['order_purchase_timestamp_x']))
ts = ts[['payment_value', 'customer_state']]

In [ ]:
ts = ts.groupby('customer_state').resample('D').sum()

In [ ]:
fig = px.line(ts.reset_index(),
              x='order_purchase_timestamp_x',
              y='payment_value',
              color='customer_state',
              title='CA par jour et par région')
fig.show()

In [ ]:
ts = orders.set_index(pd.to_datetime(orders['order_purchase_timestamp_x']))
ts = ts[['payment_value', 'customer_state']]

In [ ]:
ts = ts.groupby('customer_state').resample('D').sum()

In [ ]:
ts = ts.groupby('customer_state').cumsum().reset_index()

In [ ]:
fig = px.line(ts.reset_index(),
              x='order_purchase_timestamp_x',
              y='payment_value',
              color='customer_state',
              title='Cumul CA par région')
fig.show()

## Vendeurs et frais de port

In [ ]:
sellers = pd.merge(
    pd.merge(data['orders'], data['order_items'],
             left_on='order_id', right_on='order_id'),
    pd.merge(data['order_items'], data['sellers'],
             left_on='seller_id', right_on='seller_id'),
    left_on='order_id', right_on='order_id'
)

In [ ]:
distplot(sellers['freight_value_x'])
ax = plt.gca()
ax.set_title('Distribution des frais de port')
ax.set_xlabel('Montant frais de port')
plt.show()

In [ ]:
distplot(sellers['price_x'])
ax = plt.gca()
ax.set_title("Distribution des prix des articles")
plt.show()

In [ ]:
sellers.groupby('seller_state').sum()

In [ ]:
f, ax = plt.subplots(1, figsize=(12, 8))
ax.barh(sellers.groupby('seller_state').sum()['price_x'].sort_values().index.values,
        sellers.groupby('seller_state').sum()['price_x'].sort_values().values)
ax.set_title('Total CA réalisé par les vendeurs en fonction de la région')
plt.show()

In [ ]:
merged = pd.merge(
    pd.merge(data['customers'], data['orders'],
             left_on='customer_id', right_on='customer_id'),
    pd.merge(data['orders'], sellers,
             left_on='order_id', right_on='order_id'),
    left_on='order_id',
    right_on='order_id'
)

In [ ]:
merged

In [ ]:
distplot(merged['freight_value_x'] / merged['price_x'] * 100)
ax = plt.gca()
ax.set_xlabel('% frais de port par article')
plt.show()

### Distance entre vendeurs et acheteurs

In [ ]:
merged = pd.merge(merged, mean_loc_df.reset_index(),
                  left_on='customer_unique_id',
                  right_on='customer_unique_id')

In [ ]:
merged.rename(columns={'geolocation_lat': 'cust_lat',
                       'geolocation_lng': 'cust_lng'}, inplace=True)

In [ ]:
merged = pd.merge(merged, seller_mean_loc_df.reset_index(),
                  left_on='seller_id_x', right_on='seller_id')

In [ ]:
merged.rename(columns={"geolocation_lat": "seller_lat",
                       "geolocation_lng": "seller_lng"}, inplace=True)

In [ ]:
gdf = gpd.GeoDataFrame(merged,
                       geometry=gpd.points_from_xy(merged['cust_lat'],
                                                   merged['cust_lng']))
gdf.crs = "WGS84"
gdf = gdf.to_crs(epsg=3395)

In [ ]:
gdf['pos_seller'] = gpd.points_from_xy(
    merged['seller_lat'], merged['seller_lng'])

In [ ]:
pos_seller = gpd.GeoSeries(gdf['pos_seller'])
# set lat/long coordinate as CRS
pos_seller.crs = "WGS84"
# reprojection to mercator to get meters as units
pos_seller = pos_seller.to_crs(epsg=3395)

In [ ]:
pos_seller.crs

In [ ]:
# We can calculate the distance directly in meters
# and finally get the results in kilometers by using mercator's projection.
merged['distance'] = gdf.geometry.distance(pos_seller) / 1e3
fig = distplot(merged['distance'])
ax = plt.gca()
ax.set_title('Distance entre acheteurs et vendeurs en Km')
plt.show()

In [ ]:
sns.relplot(data=merged, x='distance', y='freight_value_x')

In [ ]:
merged = pd.merge(merged, data['products'],
                  left_on='product_id_x', right_on='product_id')

In [ ]:
sns.relplot(data=merged, x='product_weight_g', y='freight_value_x')

In [ ]:
merged['W'] = (merged['product_weight_g'] / 1e3) * merged['distance']

In [ ]:
sns.relplot(data=merged, x='W', y='freight_value_x')

In [ ]:
merged['volume'] = (merged['product_length_cm'] / 100) \
    * (merged['product_height_cm'] / 100) \
    * (merged['product_width_cm'] / 100)

In [ ]:
sns.relplot(data=merged, x='volume', y='freight_value_x')

In [ ]:
merged['vol_km'] = merged['volume'] * merged['distance']

In [ ]:
sns.relplot(data=merged, x='vol_km', y='freight_value_x')